In [638]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [639]:
import sys
sys.path.insert(0, '..//')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from e2slib.utillib import dummy_data, functions
from src.data import data_preparation, manipulations
from e2slib.structures import enums, datetime_schema
from datetime import datetime

In [640]:
demand = dummy_data.create_dummy_site_demand(2022)
carbon = dummy_data.generate_dummy_carbon_profile(2022)
price = dummy_data.generate_dummy_price_profile(2022)

In [641]:
functions.add_time_features(demand)

,Site energy [kWh],Hour,Day of week,Day of year,Month,Year,Weekday flag,Half-hour,Date,Week,season,season_number
2022-01-01 00:00:00+00:00,2,0,5,1,1,2022,weekend,0,2022-01-01,52,WINTER,1
2022-01-01 00:30:00+00:00,0,0,5,1,1,2022,weekend,1,2022-01-01,52,WINTER,1
2022-01-01 01:00:00+00:00,9,1,5,1,1,2022,weekend,2,2022-01-01,52,WINTER,1
2022-01-01 01:30:00+00:00,7,1,5,1,1,2022,weekend,3,2022-01-01,52,WINTER,1
2022-01-01 02:00:00+00:00,10,2,5,1,1,2022,weekend,4,2022-01-01,52,WINTER,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 21:30:00+00:00,7,21,5,365,12,2022,weekend,43,2022-12-31,52,WINTER,1
2022-12-31 22:00:00+00:00,2,22,5,365,12,2022,weekend,44,2022-12-31,52,WINTER,1
2022-12-31 22:30:00+00:00,4,22,5,365,12,2022,weekend,45,2022-12-31,52,WINTER,1
2022-12-31 23:00:00+00:00,7,23,5,365,12,2022,weekend,46,2022-12-31,52,WINTER,1


In [642]:
cols = [
        datetime_schema.DateTimeSchema.DAYOFWEEK,
        datetime_schema.DateTimeSchema.HALFHOUR
    ]

In [643]:
demand.columns[0]

'Site energy [kWh]'

In [644]:
df = manipulations.create_seasonal_average_week(enums.Season.SUMMER, demand)
df

,Site energy [kWh]
0.000000,3.153846
0.020833,4.153846
0.041667,6.615385
0.062500,5.076923
0.083333,4.538462
...,...
6.895833,5.230769
6.916667,3.307692
6.937500,4.923077
6.958333,4.307692


In [645]:
import plotly.express as px

# df = px.data.gapminder().query("country=='Canada'")


fig = px.line(df, x=df.index, y="Site energy [kWh]", title='Site demand for a week')
fig.show()


In [646]:
data = {'A': [10, 15, 12, 14, 20, 8, 11, 13, 16, 5, 1000, 10, 15, 12, 14, 20, 8, 11, 13, 16, 5, 1000, 10, 15, 12, 14, 20, 8, 11, 13, 16, 5, 1000],
        'B': [50, 55, 52, 54, 60, 48, 51, 53, 56, 45, 2000, 50, 55, 52, 54, 60, 48, 51, 53, 56, 45, 2000, 50, 55, 52, 54, 60, 48, 51, 53, 56, 45, 2000],
        'C': [100, 150, 120, 140, 200, 80, 110, 130, 160, 50, 300, 100, 150, 120, 140, 200, 80, 110, 130, 160, 50, 300, 100, 150, 120, 140, 200, 80, 110, 130, 160, 50, 300]}

df = pd.DataFrame(data)

# Add outliers
df.loc[3, 'A'] = 500
df.loc[6, 'B'] = 3000
df.loc[9, 'C'] = 5000

df = np.array(df)

In [647]:
# df = demand

In [648]:
# print(df)
outlier_remover = data_preparation.OutlierRemover(df)
outliers_removed_df = outlier_remover.data_cleaner()

print("\nDataFrame with outliers removed:")
print(outliers_removed_df)


DataFrame with outliers removed:
[[ 10.  50. 100.]
 [ 15.  55. 150.]
 [ 12.  52. 120.]
 [ nan  54. 140.]
 [ 20.  60. 200.]
 [  8.  48.  80.]
 [ 11.  nan 110.]
 [ 13.  53. 130.]
 [ 16.  56. 160.]
 [  5.  45.  nan]
 [ nan  nan  nan]
 [ 10.  50. 100.]
 [ 15.  55. 150.]
 [ 12.  52. 120.]
 [ 14.  54. 140.]
 [ 20.  60. 200.]
 [  8.  48.  80.]
 [ 11.  51. 110.]
 [ 13.  53. 130.]
 [ 16.  56. 160.]
 [  5.  45.  50.]
 [ nan  nan  nan]
 [ 10.  50. 100.]
 [ 15.  55. 150.]
 [ 12.  52. 120.]
 [ 14.  54. 140.]
 [ 20.  60. 200.]
 [  8.  48.  80.]
 [ 11.  51. 110.]
 [ 13.  53. 130.]
 [ 16.  56. 160.]
 [  5.  45.  50.]
 [ nan  nan  nan]]


In [649]:
fillMissing = data_preparation.FillMissingData(outliers_removed_df)
filled_na_df = fillMissing.data_cleaner(func='fillna')
filled_na_df

array([[ 10.        ,  50.        , 100.        ],
       [ 15.        ,  55.        , 150.        ],
       [ 12.        ,  52.        , 120.        ],
       [ 12.34482759,  54.        , 140.        ],
       [ 20.        ,  60.        , 200.        ],
       [  8.        ,  48.        ,  80.        ],
       [ 11.        ,  52.44827586, 110.        ],
       [ 13.        ,  53.        , 130.        ],
       [ 16.        ,  56.        , 160.        ],
       [  5.        ,  45.        , 126.55172414],
       [ 12.34482759,  52.44827586, 126.55172414],
       [ 10.        ,  50.        , 100.        ],
       [ 15.        ,  55.        , 150.        ],
       [ 12.        ,  52.        , 120.        ],
       [ 14.        ,  54.        , 140.        ],
       [ 20.        ,  60.        , 200.        ],
       [  8.        ,  48.        ,  80.        ],
       [ 11.        ,  51.        , 110.        ],
       [ 13.        ,  53.        , 130.        ],
       [ 16.        ,  56.     

In [650]:
addDatetime = data_preparation.GenerateDatetime(filled_na_df)
# filled_na_df[:, 0] = addDatetime.data_cleaner(start_date=datetime(2022,1,1), freq='D').index

In [651]:
filled_na_df = addDatetime.data_cleaner(start_date=datetime(2022,1,1), freq='D')

In [652]:
filled_na_df

array([[1.64099520e+18, 1.00000000e+01, 5.00000000e+01, 1.00000000e+02],
       [1.64108160e+18, 1.50000000e+01, 5.50000000e+01, 1.50000000e+02],
       [1.64116800e+18, 1.20000000e+01, 5.20000000e+01, 1.20000000e+02],
       [1.64125440e+18, 1.23448276e+01, 5.40000000e+01, 1.40000000e+02],
       [1.64134080e+18, 2.00000000e+01, 6.00000000e+01, 2.00000000e+02],
       [1.64142720e+18, 8.00000000e+00, 4.80000000e+01, 8.00000000e+01],
       [1.64151360e+18, 1.10000000e+01, 5.24482759e+01, 1.10000000e+02],
       [1.64160000e+18, 1.30000000e+01, 5.30000000e+01, 1.30000000e+02],
       [1.64168640e+18, 1.60000000e+01, 5.60000000e+01, 1.60000000e+02],
       [1.64177280e+18, 5.00000000e+00, 4.50000000e+01, 1.26551724e+02],
       [1.64185920e+18, 1.23448276e+01, 5.24482759e+01, 1.26551724e+02],
       [1.64194560e+18, 1.00000000e+01, 5.00000000e+01, 1.00000000e+02],
       [1.64203200e+18, 1.50000000e+01, 5.50000000e+01, 1.50000000e+02],
       [1.64211840e+18, 1.20000000e+01, 5.20000000e

In [653]:
manipulations.ResampleManipulator(filled_na_df).data_formatter('M', 'sum')

array([[ 390.03448276, 1633.34482759, 3999.65517241],
       [  17.34482759,   97.44827586,  176.55172414]])

In [654]:
added_time_features = manipulations.AddTimeFeatureManipulator(filled_na_df).data_formatter()
added_time_features

array([[Timestamp('2022-01-01 00:00:00'), 10.0, 50.0, 100.0, 0, 5, 1, 1,
        2022, 'weekend', 0, datetime.date(2022, 1, 1), 52, 'WINTER', 1],
       [Timestamp('2022-01-02 00:00:00'), 15.0, 55.0, 150.0, 0, 6, 2, 1,
        2022, 'weekend', 0, datetime.date(2022, 1, 2), 52, 'WINTER', 1],
       [Timestamp('2022-01-03 00:00:00'), 12.0, 52.0, 120.0, 0, 0, 3, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 3), 1, 'WINTER', 1],
       [Timestamp('2022-01-04 00:00:00'), 12.344827586206897, 54.0,
        140.0, 0, 1, 4, 1, 2022, 'weekday', 0, datetime.date(2022, 1, 4),
        1, 'WINTER', 1],
       [Timestamp('2022-01-05 00:00:00'), 20.0, 60.0, 200.0, 0, 2, 5, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 5), 1, 'WINTER', 1],
       [Timestamp('2022-01-06 00:00:00'), 8.0, 48.0, 80.0, 0, 3, 6, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 6), 1, 'WINTER', 1],
       [Timestamp('2022-01-07 00:00:00'), 11.0, 52.44827586206897, 110.0,
        0, 4, 7, 1, 2022, 'weekday

In [655]:
# manipulations.GroupbyManipulator(added_time_features).data_formatter(groupby=['Weekday flag'], agg='mean', target='A')

In [657]:
manipulations.EquationManipulator(added_time_features).data_formatter(1, '/ 20', 'A/20')

array([[Timestamp('2022-01-01 00:00:00'), 10.0, 50.0, 100.0, 0, 5, 1, 1,
        2022, 'weekend', 0, datetime.date(2022, 1, 1), 52, 'WINTER', 1,
        0.5],
       [Timestamp('2022-01-02 00:00:00'), 15.0, 55.0, 150.0, 0, 6, 2, 1,
        2022, 'weekend', 0, datetime.date(2022, 1, 2), 52, 'WINTER', 1,
        0.75],
       [Timestamp('2022-01-03 00:00:00'), 12.0, 52.0, 120.0, 0, 0, 3, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 3), 1, 'WINTER', 1,
        0.6],
       [Timestamp('2022-01-04 00:00:00'), 12.344827586206897, 54.0,
        140.0, 0, 1, 4, 1, 2022, 'weekday', 0, datetime.date(2022, 1, 4),
        1, 'WINTER', 1, 0.6172413793103448],
       [Timestamp('2022-01-05 00:00:00'), 20.0, 60.0, 200.0, 0, 2, 5, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 5), 1, 'WINTER', 1,
        1.0],
       [Timestamp('2022-01-06 00:00:00'), 8.0, 48.0, 80.0, 0, 3, 6, 1,
        2022, 'weekday', 0, datetime.date(2022, 1, 6), 1, 'WINTER', 1,
        0.4],
       [Timestamp('202